In [303]:
import pandas as pd
import numpy as np

# Import des datasets

## Dataset universitaire de base (toutes les années universitaires)

In [304]:
df_uni_base = pd.read_csv('../nba_forecast/data/df_csv_no_url.csv')

## Dataset des performences athletics

In [305]:
df_perf_athletics = pd.read_csv('../nba_forecast/data/perf_athletics.csv')

## Dataset universitaire (dernière année)

In [306]:
df_uni_last_year = pd.read_csv('../nba_forecast/data/last_ncaa_season.csv')

## Dataset des joueurs universitaires sans performences athlétiques avec leur position

In [307]:
string_uni_players_without_athletics  = "A.J. Hammons: C, Aaron White: PF, Abdel Nader: SG, Alex Oriakhi: PF,\
Andrew Wiggins: SF, Anthony Bennett: PF,\
Arsalan Kazemi: SF, Bam Adebayo: C, Ben Simmons: PF, Bernard James: C, Brandon Ingram: SF, C.J. McCollum: SG, Cady Lalanne: C,\
Chandler Hutchinson: SF, Darrun Hilliard: SG, De'Anthony Melton: PG, Deandre Ayton: C, Dejounte Murray: PG, Dennis Smith Jr: PG,\
Devonte' Graham: PG, Domantas Sabonis: C, Frank Mason: PG, Gary Trent Jr.: PG, Glen Rice Jr: SG, Jabari Bird: SG,\
Jabari Parker: PF, Jahlil Okafor: C, Jamal Murray: PG, Jaren Jackson Jr. : PF, Jarred Vanderbilt: PF, Jayson Tatum: SF,\
Jeff Taylor: SF, Joe Harris: SF, Joe Young: PG, Joel Embiid: C, Jonah Bolden: C, Jonathan Isaac: PF, Josh Huestis: SG,\
Josh Jackson: SF, Josh Richardson: SG, Justin Hamilton: C, Karl-Anthony Towns: C, Lauri Markkanen: PF, Lonzo Ball: PG,\
Malik Monk: SG, Marvin Bagley III : PF, Michael Porter Jr. : SF, Mikal Bridges: SF, Mitch McGary: PF, Ray Spalding: PF,\
Robert Sacre: C, Robert Williams: C, Romero Osby: SF, Roy Devyn Marble: Missing, Sir 'Dominic Pointer: SF,\
Stephen Zimmerman Jr. : C, Sterling Brown: SG, Tyrone Wallace : PG, Vince Edwards: SF, Wendell Carter Jr. : C"

In [308]:
list_string_uni_players_without_athletics = [list(map(lambda word:word.strip(), x.split(':'))) \
                                               for x in string_uni_players_without_athletics.split(',')]

In [309]:
list_player_name = np.array(list_string_uni_players_without_athletics)[:,:1].reshape(-1)

In [310]:
list_position = np.array(list_string_uni_players_without_athletics)[:,1:].ravel()

In [311]:
dico_uni_players_without_athletics = {'player_name':list_player_name,'position':list_position}

In [312]:
df_uni_players_without_athletics  = pd.DataFrame(dico_uni_players_without_athletics)

In [313]:
df_uni_players_without_athletics.head()

,player_name,position
0,A.J. Hammons,C
1,Aaron White,PF
2,Abdel Nader,SG
3,Alex Oriakhi,PF
4,Andrew Wiggins,SF


# Vérification sur les Datasets


In [315]:
df_uni_base['player_name'].nunique()

334

In [316]:
df_uni_last_year['player_name'].nunique()

334

# Correspondance des noms des joueurs entre le Dataset universitaire ( dernière année) et le Dataset performences athlétics

## Noms à corriger dans le Dataset *df_perf_athletics*

In [317]:
df_perf_athletics[df_perf_athletics['PLAYER_NAME'].str.contains('[^a-zA-Z0-9^\s]', regex=True)]['PLAYER_NAME']

4                 J'Covan Brown
23          Darius Johnson-Odom
28       Michael Kidd-Gilchrist
43                 Kyle O'Quinn
66     Kentavious Caldwell-Pope
69      Michael Carter-Williams
83             Tim Hardaway Jr.
143                   C.J. Fair
146               P.J. Hairston
157              K.J. McDaniels
161         Johnny O?Bryant III
175                 T.J. Warren
176                 C.J. Wilcox
186         Willie Cauley-Stein
201     Rondae Hollis-Jefferson
204                 R.J. Hunter
225            D?Angelo Russell
228                 J.P. Tokoto
257         Dorian Finney-Smith
302                V.J. Beachem
318                De?Aaron Fox
326        Justin Jackson (UMD)
327        Justin Jackson (UNC)
352          Derrick Walton Jr.
355         Nigel Williams-Goss
356                 D.J. Wilson
364            Keita Bates-Diop
382     Shai Gilgeous-Alexander
Name: PLAYER_NAME, dtype: object

In [319]:
df_perf_athletics[df_perf_athletics['PLAYER_NAME'].str.contains('Devonte', regex=True)]['PLAYER_NAME']

383    Devonte Graham
Name: PLAYER_NAME, dtype: object

In [320]:
df_perf_athletics.loc[161,'PLAYER_NAME'] = "Johnny O'Bryant"

In [321]:
df_perf_athletics.loc[225,'PLAYER_NAME'] = "D'Angelo Russell"

In [322]:
df_perf_athletics.loc[318,'PLAYER_NAME'] = "De'Aaron Fox"

## Noms à corriger dans le Dataset *df_uni_last_year*

In [323]:
df_uni_last_year[df_uni_last_year['player_name'].str.contains('Justin Jackson', regex=False)]['player_name']

259    Justin Jackson
335    Justin Jackson
Name: player_name, dtype: object

In [324]:
df_uni_last_year.iloc[[259,335]]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
259,Justin Jackson,2016-17,UNC,ACC,40,39,1281,21.3,0.555,0.531,...,9.5,25.7,4.3,1.8,6.1,0.191,7.7,2.1,9.8,3
335,Justin Jackson,2017-18,Maryland,Big Ten,11,10,321,14.2,0.470,0.416,...,21.8,24.1,0.0,0.6,0.7,0.083,0.2,1.7,1.9,2


In [325]:
df_uni_last_year.loc[259, 'player_name'] = 'Justin Jackson (UNC)'

In [326]:
df_uni_last_year.loc[335, 'player_name'] = 'Justin Jackson (UMD)'

# Concaténation de *df_perf_athletics* & *df_uni_players_without_athletics* et traitement des doublons

Le nom des joueurs (colonne 'player_name') est la clé utilisée pour merger l'ensemble des Datasets

## Traitement préalable sur les Datasets (drop columns inutiles)

In [328]:
df_perf_athletics.drop(columns='Unnamed: 0', inplace=True)

In [329]:
df_perf_athletics.rename(columns={'PLAYER_NAME': 'player_name', 'POSITION': 'position'}, inplace=True)

## Concaténation de *df_perf_athletics* et *df_uni_players_without_athletics* 

In [330]:
df_athletics = pd.concat([df_perf_athletics,df_uni_players_without_athletics], axis=0, ignore_index=True)

In [331]:
df_athletics

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
0,2012-13,Quincy Acy,PF,7.10%,9.50,11.00,6' 6.5'',6' 7.75'',8' 10.5'',223.8,7' 2.75''
1,2012-13,Harrison Barnes,SF-SG,9.60%,9.00,8.50,6' 7'',6' 8'',8' 5.5'',227.8,6' 11.25''
2,2012-13,Will Barton,SG-SF,4.10%,9.00,9.50,6' 5'',6' 6'',8' 6.5'',174.4,6' 9.75''
3,2012-13,Bradley Beal,SG,6.00%,8.50,9.00,6' 3.25'',6' 4.75'',8' 4'',201.8,6' 8''
4,2012-13,J'Covan Brown,PG,12.50%,7.75,8.25,6' 1'',6' 2.25'',8' 0'',202.4,6' 5.25''
...,...,...,...,...,...,...,...,...,...,...,...
482,NaN,Stephen Zimmerman Jr.,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
483,NaN,Sterling Brown,SG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,NaN,Tyrone Wallace,PG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,NaN,Vince Edwards,SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [332]:
df_athletics['player_name'].nunique()

472

## Identification des doublons, traitement au cas par cas

In [333]:
df_athletics[df_athletics['player_name'].duplicated()]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
100,2013-14,Tony Mitchell,PF-SF,5.55%,9.00,10.00,6' 7.5'',6' 8.75'',8' 10.5'',236.4,7' 2.5''
304,2017-18,Jaron Blossomgame,SF,5.70%,8.75,10.25,6' 5.75'',6' 6.75'',8' 8.5'',218.6,6' 10''
321,2017-18,Josh Hart,SG-SF,5.80%,8.50,9.50,6' 3.75'',6' 5'',8' 5'',209.2,6' 8.25''
322,2017-18,Nigel Hayes,PF,11.50%,8.75,10.00,6' 6.25'',6' 7.5'',8' 8.5'',254.2,7' 3.25''
348,2017-18,Caleb Swanigan,PF,-%,9.50,10.25,6' 7.5'',6' 8.5'',9' 0'',245.6,7' 3''
350,2017-18,Melo Trimble,PG,10.60%,8.00,8.75,6' 1.25'',6' 2.5'',7' 11.5'',195.2,6' 2''
358,2018-19,Rawle Alkins,SG,8.90%,8.50,10.00,6' 2.75'',6' 4.25'',8' 3'',217.4,6' 8.75''
375,2018-19,Hamidou Diallo,SG,4.45%,8.75,9.00,6' 4.25'',6' 6'',8' 7'',197.0,6' 11.5''
391,2018-19,Justin Jackson,PF,8.90%,9.00,9.50,6' 6'',6' 6.75'',8' 9'',229.4,7' 3''
404,2018-19,Malik Newman,SG,6.25%,8.50,9.50,6' 2.5'',6' 3.25'',8' 2.5'',189.2,6' 5.5''


In [334]:
df_athletics[df_athletics['player_name'] == 'Justin Jackson']

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
263,2016-17,Justin Jackson,SF,7.60%,8.75,8.75,6' 7'',6' 8.25'',8' 7.5'',193.4,6' 10.75''
391,2018-19,Justin Jackson,PF,8.90%,9.00,9.50,6' 6'',6' 6.75'',8' 9'',229.4,7' 3''


In [335]:
df_athletics[df_athletics['player_name'].str.contains('Justin Jackson', regex=False)]['player_name']

263          Justin Jackson
326    Justin Jackson (UMD)
327    Justin Jackson (UNC)
391          Justin Jackson
Name: player_name, dtype: object

In [336]:
df_athletics.iloc[[263,326,327,391]]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
263,2016-17,Justin Jackson,SF,7.60%,8.75,8.75,6' 7'',6' 8.25'',8' 7.5'',193.4,6' 10.75''
326,2017-18,Justin Jackson (UMD),SF,7.10%,9.00,9.50,6' 5.75'',6' 7'',8' 11'',219.4,7' 3.25''
327,2017-18,Justin Jackson (UNC),SF,8.10%,8.75,9.25,6' 7'',6' 8.25'',8' 8.5'',200.8,6' 11''
391,2018-19,Justin Jackson,PF,8.90%,9.00,9.50,6' 6'',6' 6.75'',8' 9'',229.4,7' 3''


In [339]:
df_athletics.drop([263,391], inplace=True)

In [340]:
df_athletics[df_athletics['player_name'].str.contains('Justin Jackson', regex=False)]['player_name']

326    Justin Jackson (UMD)
327    Justin Jackson (UNC)
Name: player_name, dtype: object

## Drop des doublons identifiés

In [211]:
df_athletics[df_athletics['player_name'].duplicated()]['player_name'].values

array(['Tony Mitchell', 'Jaron Blossomgame', 'Josh Hart', 'Nigel Hayes',
       'Caleb Swanigan', 'Melo Trimble', 'Rawle Alkins', 'Hamidou Diallo',
       'Justin Jackson', 'Malik Newman', 'Moritz Wagner', 'Bernard James',
       'Jeff Taylor', 'Joe Harris', 'Ray Spalding'], dtype=object)

### Doublons identifiés

In [341]:
df_athletics[df_athletics['player_name'].duplicated()]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
100,2013-14,Tony Mitchell,PF-SF,5.55%,9.00,10.00,6' 7.5'',6' 8.75'',8' 10.5'',236.4,7' 2.5''
304,2017-18,Jaron Blossomgame,SF,5.70%,8.75,10.25,6' 5.75'',6' 6.75'',8' 8.5'',218.6,6' 10''
321,2017-18,Josh Hart,SG-SF,5.80%,8.50,9.50,6' 3.75'',6' 5'',8' 5'',209.2,6' 8.25''
322,2017-18,Nigel Hayes,PF,11.50%,8.75,10.00,6' 6.25'',6' 7.5'',8' 8.5'',254.2,7' 3.25''
348,2017-18,Caleb Swanigan,PF,-%,9.50,10.25,6' 7.5'',6' 8.5'',9' 0'',245.6,7' 3''
350,2017-18,Melo Trimble,PG,10.60%,8.00,8.75,6' 1.25'',6' 2.5'',7' 11.5'',195.2,6' 2''
358,2018-19,Rawle Alkins,SG,8.90%,8.50,10.00,6' 2.75'',6' 4.25'',8' 3'',217.4,6' 8.75''
375,2018-19,Hamidou Diallo,SG,4.45%,8.75,9.00,6' 4.25'',6' 6'',8' 7'',197.0,6' 11.5''
404,2018-19,Malik Newman,SG,6.25%,8.50,9.50,6' 2.5'',6' 3.25'',8' 2.5'',189.2,6' 5.5''
420,2018-19,Moritz Wagner,C-PF,6.75%,9.00,10.75,6' 10.5'',6' 11.5'',9' 0'',241.4,7' 0''


### Drop des doublons

In [372]:
df_athletics[df_athletics['player_name'].str.contains('Ray Spalding', regex=False)]['player_name']

415    Ray Spalding
476    Ray Spalding
Name: player_name, dtype: object

In [373]:
df_athletics.loc[[415,476]]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
415,2018-19,Ray Spalding,PF,5.55%,9.25,9.00,6' 9'',6' 10.25'',9' 1'',215.4,7' 4.75''
476,NaN,Ray Spalding,PF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [374]:
df_athletics.drop([244], inplace=True) #Jaron Blossomgame
df_athletics.drop([260], inplace=True) #Josh Hart
df_athletics.drop([261], inplace=True) #Nigel Hayes
df_athletics.drop([348], inplace=True) #Caleb Swanigan
df_athletics.drop([286], inplace=True) #Melo Trimble
df_athletics.drop([295], inplace=True) #Rawle Alkins
df_athletics.drop([312], inplace=True) #Hamidou Diallo
df_athletics.drop([273], inplace=True) #Malik Newman
df_athletics.drop([351], inplace=True) #Moritz Wagner
df_athletics.drop([436], inplace=True) #Bernard James
df_athletics.drop([458], inplace=True) #Jeff Taylor
df_athletics.drop([459], inplace=True) #Joe Harris
df_athletics.drop([476], inplace=True) #Ray Spalding

## Identification des doublons non repérés à cause d'une différence sur l'orthographe

In [212]:
df_athletics[df_athletics['player_name'].str.contains('[^a-zA-Z0-9^\s]', regex=True)]['player_name']

4                 J'Covan Brown
23          Darius Johnson-Odom
28       Michael Kidd-Gilchrist
43                 Kyle O'Quinn
66     Kentavious Caldwell-Pope
69      Michael Carter-Williams
83             Tim Hardaway Jr.
143                   C.J. Fair
146               P.J. Hairston
157              K.J. McDaniels
161             Johnny O'Bryant
175                 T.J. Warren
176                 C.J. Wilcox
186         Willie Cauley-Stein
201     Rondae Hollis-Jefferson
204                 R.J. Hunter
225            D'Angelo Russell
228                 J.P. Tokoto
257         Dorian Finney-Smith
302                V.J. Beachem
318                De'Aaron Fox
326        Justin Jackson (UMD)
327        Justin Jackson (UNC)
352          Derrick Walton Jr.
355         Nigel Williams-Goss
356                 D.J. Wilson
364            Keita Bates-Diop
382     Shai Gilgeous-Alexander
427                A.J. Hammons
438               C.J. McCollum
442           De'Anthony Melton
446     

In [214]:
df_athletics[df_athletics['player_name'].str.contains('Devonte', regex=True)]['player_name']

383     Devonte Graham
446    Devonte' Graham
Name: player_name, dtype: object

In [215]:
df_athletics[df_athletics['player_name'].str.contains('Dominic Pointer', regex=True)]['player_name']

481    Sir 'Dominic Pointer
Name: player_name, dtype: object

In [216]:
df_athletics[df_athletics['player_name'].str.contains('Anthony Melton', regex=True)]['player_name']

400     DeAnthony Melton
442    De'Anthony Melton
Name: player_name, dtype: object

In [221]:
df_athletics[df_athletics['player_name'].str.contains('Aaron', regex=True)]['player_name']

136       Aaron Craft
144      Aaron Gordon
197    Aaron Harrison
318      De'Aaron Fox
388     Aaron Holiday
428       Aaron White
Name: player_name, dtype: object

In [222]:
df_athletics[df_athletics['player_name'].str.contains('Hammons', regex=True)]['player_name']

427    A.J. Hammons
Name: player_name, dtype: object

In [375]:
df_athletics.loc[[383,446,400,442]]

,year,player_name,position,BODY_FAT_PCT,HAND_LENGTH,HAND_WIDTH,HEIGHT_WO_SHOES,HEIGHT_W_SHOES,STANDING_REACH,WEIGHT,WINGSPAN
383,2018-19,Devonte Graham,PG,4.80%,8.50,9.50,6' 0.25'',6' 1.5'',8' 0'',186.4,6' 6.25''
446,NaN,Devonte' Graham,PG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400,2018-19,DeAnthony Melton,SG-PG,5.20%,8.50,9.50,6' 2.25'',6' 3.25'',8' 3.5'',193.2,6' 8.5''
442,NaN,De'Anthony Melton,PG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Doublons à effacer : Devonte Graham, DeAnthony Melton 

In [377]:
df_athletics.drop([446,442], inplace=True)

Le seul nom dupliqué qui reste est *Tony Mitchell* qui correspond cependant à deux joueurs différents.

In [380]:
df_athletics['player_name'].duplicated().sum()

1

 # Merging de *df_uni_last_year* et de *df_athletics*

## Traitement des doublons dans df_uni_last_year

In [384]:
df_uni_last_year[df_uni_last_year['player_name'].duplicated()]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
26,Arnett Moultrie,2011-12,Mississippi State,SEC,30,30,1073,25.2,0.610,0.561,...,15.3,23.3,3.6,1.4,5.0,0.186,4.8,0.4,5.2,1
35,Orlando Johnson,2011-12,UCSB,Big West,31,31,1067,26.0,0.560,0.527,...,12.2,30.5,4.0,1.2,5.2,0.194,6.5,-0.2,6.3,3
45,Justin Hamilton,2011-12,LSU,SEC,33,32,990,22.7,0.553,0.498,...,12.9,22.6,2.6,1.7,4.3,0.172,3.8,2.1,5.9,1
94,Romero Osby,2012-13,Oklahoma,Big 12,32,31,912,28.9,0.604,0.539,...,9.0,26.0,4.2,1.4,5.6,0.244,7.8,2.1,10.0,2
97,Colton Iverson,2012-13,Colorado State,MWC,35,35,1033,27.4,0.602,0.596,...,14.3,23.8,3.9,1.8,5.7,0.219,6.4,1.5,7.9,1
99,Arsalan Kazemi,2012-13,Oregon,Pac-12,35,27,1016,24.4,0.627,0.595,...,16.6,15.2,2.7,3.0,5.7,0.226,5.3,5.4,10.8,1
102,Alex Oriakhi,2012-13,Missouri,SEC,34,34,878,27.2,0.674,0.639,...,14.2,18.5,3.4,1.6,5.1,0.230,5.7,2.7,8.4,1
123,Rodney Hood,2013-14,Duke,ACC,35,33,1150,20.1,0.590,0.550,...,10.2,23.8,3.6,1.2,4.9,0.170,6.3,0.9,7.2,1
142,Jordan Clarkson,2013-14,Missouri,SEC,35,35,1228,20.2,0.547,0.482,...,14.3,27.9,3.2,1.2,4.4,0.142,3.8,0.5,4.3,1
151,Xavier Thames,2013-14,San Diego State,MWC,36,36,1126,26.5,0.557,0.478,...,8.2,28.6,4.7,2.8,7.5,0.267,7.2,5.2,12.4,3


In [385]:
df_uni_last_year[df_uni_last_year['player_name'].str.contains('Arnett Moultrie', regex=True)]['player_name']

25    Arnett Moultrie
26    Arnett Moultrie
Name: player_name, dtype: object

In [386]:
df_uni_last_year.loc[[25,26]]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
25,Arnett Moultrie,2009-10,UTEP,CUSA,33,33,955,16.2,0.515,0.491,...,18.2,20.2,0.8,2.3,3.1,0.131,NaN,NaN,NaN,2
26,Arnett Moultrie,2011-12,Mississippi State,SEC,30,30,1073,25.2,0.610,0.561,...,15.3,23.3,3.6,1.4,5.0,0.186,4.8,0.4,5.2,1


In [387]:
df_uni_last_year[df_uni_last_year['player_name'].str.contains('Orlando Johnson', regex=True)]['player_name']

34    Orlando Johnson
35    Orlando Johnson
Name: player_name, dtype: object

In [388]:
df_uni_last_year.loc[[34,35]]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
34,Orlando Johnson,2007-08,Loyola Marymount,WCC,31,30,807,NaN,0.504,0.474,...,17.4,NaN,0.8,-0.2,0.5,0.026,NaN,NaN,NaN,1
35,Orlando Johnson,2011-12,UCSB,Big West,31,31,1067,26.0,0.560,0.527,...,12.2,30.5,4.0,1.2,5.2,0.194,6.5,-0.2,6.3,3


In [389]:
df_uni_last_year[df_uni_last_year['player_name'].str.contains('Justin Hamilton', regex=True)]['player_name']

44    Justin Hamilton
45    Justin Hamilton
Name: player_name, dtype: object

In [391]:
df_uni_last_year.loc[[44,45]]

,player_name,season,school_name,conf_abbr,g,gs,mp,per,ts_pct,efg_pct,...,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,years
44,Justin Hamilton,2009-10,Iowa State,Big 12,31,31,663,18.0,0.643,0.617,...,16.4,13.6,1.6,0.9,2.5,0.150,NaN,NaN,NaN,2
45,Justin Hamilton,2011-12,LSU,SEC,33,32,990,22.7,0.553,0.498,...,12.9,22.6,2.6,1.7,4.3,0.172,3.8,2.1,5.9,1


In [392]:
df_uni_last_year.drop_duplicates(subset='player_name', keep='last', inplace=True, ignore_index=True)

In [395]:
df_uni_last_year.duplicated().sum()

0